# SFTT (Instruction Tuning) using Unsloth

In [32]:
#install latest version of unsloth
#%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#%pip install --upgrade unsloth
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
%pip install --upgrade unsloth_zoo
%pip install wandb

Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /data/temporary/pip-req-build-8_wldsqp
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /data/temporary/pip-req-build-8_wldsqp
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173746 sha256=d24c513cc390ff39d829f5174b34357c59e8c03ef4692f8c0fba31bc8bf26730
  Stored in directory: /data/temporary/pip-ephem-wheel-cache-fmx16g_e/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
Note: you may need to restart the kernel to use updated packages.
Note: you may need to r

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset

import torch

# random state
SEED = 42

# reproducibility
## torch
torch.manual_seed(SEED)

## python
import random
random.seed(SEED)

## numpy
import numpy as np
np.random.seed(SEED)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 🧠 Load Model

In [2]:
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_id = "meta-llama/Llama-3.3-70B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.381 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

## Init LoRA

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Prepare Data

In [4]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = load_dataset("ctu-aic/ask_library_cs", split="train")

In [5]:
def standardize_ask_library(dataset):
    """Standardize dataset to hugginface conversations format"""
    # filter cs language only
    dataset = dataset.filter(lambda x: x["language"] == "cs")

    questions = dataset["question"]
    answers = dataset["answer"]


    conversations = []
    for q, a in zip(questions, answers):
        user = {"role": "user", "content": q}
        assistant = {"role": "assistant", "content": a}
        conversations.append([user, assistant])

    dataset = dataset.add_column("conversations", conversations)

    return dataset

In [6]:
dataset = standardize_ask_library(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True)

In [7]:
dataset[0]["conversations"]

[{'content': 'Dobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně zadaný pin nebo čtenářské číslo, delší období jsem knihy nevypůjčovala, chci teď v době koronaviru zkusit e-lerning. Děkuji za odpověď.',
  'role': 'user'},
 {'content': 'Vážená paní,  moc mě to mrzí, ale v databázi čtenářů Vás již nemáme. Podle Nařízení na ochranu osobních údajů nemůžeme spravovat údaje čtenářů déle než šest měsíců od propadlé registrace. Potom musíme všechny Vaše údaje zarchivovat. Nabídnout Vám mohu založení nového čtenářského konta, toto lze však učinit až v okamžiku znovuotevření knihovny.  Momentálně můžete zkusit e-knihy, které jdou stahovat zdarma, více informací získáte na tomto webu -https://protiviru.knihovny.cz/  Děkuji za pochopení a přeji pevné zdraví',
  'role': 'assistant'}]

In [8]:
dataset[0]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nDobrý den, nemohu se prosím přihlásit přes zadané údaje, hlásí to chybně zadaný pin nebo čtenářské číslo, delší období jsem knihy nevypůjčovala, chci teď v době koronaviru zkusit e-lerning. Děkuji za odpověď.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nVážená paní,  moc mě to mrzí, ale v databázi čtenářů Vás již nemáme. Podle Nařízení na ochranu osobních údajů nemůžeme spravovat údaje čtenářů déle než šest měsíců od propadlé registrace. Potom musíme všechny Vaše údaje zarchivovat. Nabídnout Vám mohu založení nového čtenářského konta, toto lze však učinit až v okamžiku znovuotevření knihovny.  Momentálně můžete zkusit e-knihy, které jdou stahovat zdarma, více informací získáte na tomto webu -https://protiviru.knihovny.cz/  Děkuji za pochopení a přeji pevné zdraví<|eot_id|>'

## Train the model

In [9]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = SEED,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/15789 [00:00<?, ? examples/s]

In [10]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/15789 [00:00<?, ? examples/s]

In [11]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBohužel jsme se dopracovali jen k protektorátu a tam jsme úplně samostatní nebyli.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDobrý den, máte pravdu, Protektorát Čechy a Morava za samostatný český stát opravdu považovat nelze. Podíváme-li se na časovou osu, tak nejblíže k\xa0naší době lze o samostatném českém státě hovořit v\xa0případě Zemí Koruny české, které byly ustanoveny Karlem IV. 7. dubna 1348 a zanikly až vznikem Československé republiky na podzim\xa0r. 1918. Nástupem Ferdinanda I. Habsburského na český trůn (23.října 1526) však o svoji bývalou mocenskou autonomii přišly (VOREL, s. 81-87), státní útvar z roku 1526 bychom tedy v\xa0případě Vaší otázky mohli považovat za nejpozdější, který přichází v úvahu. Naopak první zmínky o českém státě, který vystupoval navenek jako samostatný,

In [12]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                        \n\nDobrý den, máte pravdu, Protektorát Čechy a Morava za samostatný český stát opravdu považovat nelze. Podíváme-li se na časovou osu, tak nejblíže k\xa0naší době lze o samostatném českém státě hovořit v\xa0případě Zemí Koruny české, které byly ustanoveny Karlem IV. 7. dubna 1348 a zanikly až vznikem Československé republiky na podzim\xa0r. 1918. Nástupem Ferdinanda I. Habsburského na český trůn (23.října 1526) však o svoji bývalou mocenskou autonomii přišly (VOREL, s. 81-87), státní útvar z roku 1526 bychom tedy v\xa0případě Vaší otázky mohli považovat za nejpozdější, který přichází v úvahu. Naopak první zmínky o českém státě, který vystupoval navenek jako samostatný, jsou datovány do\xa0přemyslovské éry, tedy do doby sjednocování kmenů žijících na území Čech a následné expanze na\xa0území Moravy, tedy přibližně mezi konec 9. století a začátek 11. století. V\xa0literatuře hovoříme o\xa0tzv.\xa0Českém knížectví, případně o Přemy

In [13]:
#Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.381 GB.
2.635 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15,789 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 246
 "-____-"     Number of trainable parameters = 24,313,856
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.850800
2,2.916100
3,2.783200
4,2.789200
5,2.815600
6,2.791700
7,2.781400
8,2.646200
9,2.659600
10,2.766400


In [15]:
#Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4329.0353 seconds used for training.
72.15 minutes used for training.
Peak reserved memory = 19.932 GB.
Peak reserved memory for training = 17.297 GB.
Peak reserved memory % of max memory = 50.613 %.
Peak reserved memory for training % of max memory = 43.922 %.


## Inference

In [3]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Kdo je nejlepší fotbalista na světě?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 2048, use_cache = True,
                         temperature = 1, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKdo je nejlepší fotbalista na světě?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTato otázka je vždy aktuální a způsobuje mnoho debat. Nejlepší fotbalista na světě je subjektivní a může se lišit podle osobních preferencí, statistik a individuálních výkonů. Nicméně, podle mnoha odborníků a fanoušků, jsou mezi nejlepšími fotbalisty na světě:\n\n1. **Lionel Messi**: Argentinský útočník, který hraje za Paris Saint-Germain, je často považován za jednoho z nejlepších fotbalistů všech dob. Jeho výjimečné technické dovednosti, rychlost, driblování a střelecké schopnosti ho činí úžasným hráčem.\n2. **Cristiano Ronaldo**: Portugalský útočník, který hraje za Al-Nassr, je další velkým jménem ve fotbalovém světě. Jeho neuvěřitelná síla, rychlost, střelecké schopnosti a vedení na hřišti ho činí jedním z

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nMarie Terezie je<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDobrý den, Marie Terezie se v českém jazyce nazývá Marie Terézie, je českou královskou královnou. Pozdějí se na dvě části: Marie - z řeckého word Marie (král, prince) a Terezie - z řeckého word Terésia (král) a to v době, kdy se královna pozdívala jako královna a pozdívala se jako královna. Dále se podepisovala jako Marie Terezie, kde se pozdívala jako královna a']